In [43]:
import heapq

class Event:
    '''
    Store the properties of one event in the Schedule class defined below. Each
    event has a time at which it needs to run, a function to call when running
    the event, along with the arguments and keyword arguments to pass to that
    function.
    '''
    def __init__(self, timestamp, function, *args, **kwargs):
        self.timestamp = timestamp
        self.function = function
        self.args = args
        self.kwargs = kwargs

    def __lt__(self, other):
        '''
        This overloads the less-than operator in Python. We need it so the
        priority queue knows how to compare two events. We want events with
        earlier (smaller) times to go first.
        '''
        return self.timestamp < other.timestamp

    def run(self, schedule):
        '''
        Run an event by calling the function with its arguments and keyword
        arguments. The first argument to any event function is always the
        schedule in which events are being tracked. The schedule object can be
        used to add new events to the priority queue.
        '''
        self.function(schedule, *self.args, **self.kwargs)

In [44]:
class Schedule:
    '''
    Implement an event schedule using a priority queue. You can add events and
    run the next event.
    
    The `now` attribute contains the time at which the last event was run.
    '''
    
    def __init__(self):
        self.now = 0  # Keep track of the current simulation time
        self.priority_queue = []  # The priority queue of events to run
    
    def add_event_at(self, timestamp, function, *args, **kwargs):
        # Add an event to the schedule at a particular point in time.
        heapq.heappush(
            self.priority_queue,
            Event(timestamp, function, *args, **kwargs))
    
    def add_event_after(self, interval, function, *args, **kwargs):
        # Add an event to the schedule after a specified time interval.
        self.add_event_at(self.now + interval, function, *args, **kwargs)
    
    def next_event_time(self):
        # Return the time of the next event. The `now` attribute of this class
        # contain the time of the last event that was run.
        return self.priority_queue[0].timestamp

    def run_next_event(self):
        # Get the next event from the priority queue and run it.
        event = heapq.heappop(self.priority_queue)
        self.now = event.timestamp
        event.run(self)
        
    def __repr__(self):
        return (
            f'Schedule() at time {self.now} ' +
            f'with {len(self.priority_queue)} events in the queue')
    
    def print_events(self):
        # Print out diagnostic information about the events in the schedule.
        print(repr(self))
        for event in sorted(self.priority_queue):
            print(f'   {event.timestamp}: {event.function.__name__}')

In [45]:
import scipy.stats as sts
class Queue:
    """
    Tracking the number of people in the queue, as well as the number of people getting on the bus.It also schedules when the passenger gets on the bus and when 
    """
    def __init__(self, arrival_rate, service_rate):
        self.arrival_rate = arrival_rate
        self.service_rate = service_rate
        self.service_time = 1/service_rate
        self.len_queue = 0 
        self.passenger = 0 
        
    def new_bus_arrival(self):
        self.len_queue += 1
        print(f"{self.len_queue} people are waiting in the queue.")
    
    def new_bus_service(self,schedule):
        self.passenger += 1 
        next_arrival = sts.expon(scale=1/self.arrival_rate).rvs()
        
        if next_arrival < self.service_time:
            schedule.add_event_after(self.service_time, self.new_bus_departure)
        
        schedule.add_event_after(self.service_time, self.new_bus_departure)
        print(f"The passenger {self.passenger} is now getting on the bus.")
        
    def new_bus_departure(self, schedule):
        next_arrival = sts.expon(scale=1/self.arrival_rate).rvs(size=1)
        
        schedule.add_event_after(self.service_time, self.new_bus_service)

        print(f"The passenger {self.passenger} was on the bus {schedule.now} and is now leaving the bus.")
        

In [46]:
class BusSystem:
    """
    Contains the queues and keeps track of people arriving in the queue
    """
    def __init__(self,arrival_rate, service_rate):
        self.queue = Queue(arrival_rate, service_rate)
        
    def run(self, schedule, arrival_rate):
        arrival_time = sts.expon(scale=1/arrival_rate).rvs(size=1)    
        schedule.add_event_at(arrival_time, self.queue.new_bus_service)

In [47]:
def run_simulation(arrival_rate, service_rate, run_until):
    """
    Input: 
        Service rate, arrival rate, time
        
    Output: 
        Simulation
    """
    schedule = Schedule()
    
    busSystem = BusSystem(service_rate,arrival_rate)
    busSystem.run(schedule, arrival_rate)
    
    while schedule.next_event_time() <= run_until:
        schedule.run_next_event()

In [48]:
run_simulation(4,3,8)

The passenger 1 is now getting on the bus.
The passenger 1 was on the bus [0.28181998] and is now leaving the bus.
The passenger 1 was on the bus [0.28181998] and is now leaving the bus.
The passenger 2 is now getting on the bus.
The passenger 3 is now getting on the bus.
The passenger 3 was on the bus [0.78181998] and is now leaving the bus.
The passenger 3 was on the bus [0.78181998] and is now leaving the bus.
The passenger 3 was on the bus [0.78181998] and is now leaving the bus.
The passenger 3 was on the bus [0.78181998] and is now leaving the bus.
The passenger 4 is now getting on the bus.
The passenger 5 is now getting on the bus.
The passenger 6 is now getting on the bus.
The passenger 7 is now getting on the bus.
The passenger 7 was on the bus [1.28181998] and is now leaving the bus.
The passenger 7 was on the bus [1.28181998] and is now leaving the bus.
The passenger 7 was on the bus [1.28181998] and is now leaving the bus.
The passenger 7 was on the bus [1.28181998] and is 